In [2]:
import gym
import collections  # Deque for replay buffer
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
learning_rate = 0.005
gamma = 0.98
buffer_limit = 50000
batch_size = 32

In [42]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
        
    def put(self,transition):
        self.buffer.append(transition)
        
    def sample(self,n):
        mini_batch = random.sample(self.buffer,n) # sampling n from self.buffer
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_list = [],[],[],[],[]
        
        for transition in mini_batch:
            s,a,r,s_prime,done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            dome_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst,dtype=torch.float),torch.tensor(a_lst),torch.tensor(r_lst),\
        torch.tensor(s_prime_lst,dtype=torch.float),torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

In [43]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet,self).__init__()
        self.fc1 = nn.Linear(4,256)
        self.fc2 = nn.Linear(256,2) # output 2   left or right
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def sample_action(self,obs,epsilon): #epsilon do random 
        out = self.forward(obs)
        coin = random.random()
        if coin>epsilon:
            return random.randint(0,1)
        else:
            return out.argmax().item()

In [44]:
for n_epi in range(10000):
    epsilon = max(0.01,0.08-0.01*(n_epi/200))
    s = env.reset()
    
    for t in range(600):
        a = q.sample_action(torch.from_numpy(s).float(),epsilon)
        s_prime,r,done,info = env.step(a)
        done_mask = 0.0 if done else 1.0
        memory.put((s,a,r/100.,s_prime,done_mask))
        s = s_prime
        
        score += r
        if done:
            break
    
    if memory.size()>2000:
        train(q,q_target,memory,optimizer)

NameError: name 'env' is not defined

In [45]:
def train(q,q_target,memory,optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        
        q_out = q(s) #[32,2]shape
        q_a = q_out.gather(1,a)  # 취한 action 
        max_q_prime = q(s_prime).max(1)[0].unsqueeze(1)
        target = r+gamma*max_q_prime*done_mask
        loss = F.smooth_l1_loss(q_a,target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()